In [ ]:
import time
print(time.ctime())
%load_ext autoreload
%autoreload 2
%autosave 10


#  Create tardis data file

https://tardis-sn.github.io/tardis/io/configuration/components/models/index.html#id7

```
model:
    structure:
        type: file
        filename: density.dat
        filetype: simple_ascii
        v_inner_boundary: 11000 km/s
        v_outer_boundary: 20000 km/s
   abundances:
        type: file
        filename: abund.dat
        filetype: simple_ascii
```        

In [ ]:
# for plottong
# import matplotlib
# matplotlib.use('TkAgg')
%matplotlib inline
# %matplotlib notebook
from matplotlib import rcParams
# from matplotlib.lines import lineStyles
import matplotlib.pyplot as plt
from matplotlib import gridspec, colors
from matplotlib.patches import Rectangle
# import matplotlib.lines as mlines

if True:
    # plt.style.use('../../lib/mpl/mtl2smv2.stl')
    plt.style.use('./mtl2smv2.stl')
    
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['font.size'] = 14    

import logging

mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.ERROR)


In [ ]:
import os
import sys
import operator
import itertools

import numpy as np
# import scipy
# from scipy import stats
# from scipy import interpolate
# from scipy import optimize
import pandas as pd
# import seaborn as sns
from astropy import constants as consts
from astropy import units as u


sys.path.append(os.path.expanduser('/home/bakl/Sn/Release/python/pystella'))
import pystella as ps


In [ ]:
def fig_save(fig, fname, ext=".pdf"):
    if not fname.endswith(ext):
        fname += ext
    fsave = os.path.expanduser(fname)
    print("Save plot to %s " % fsave)
    fig.savefig(fsave, bbox_inches='tight')
    
figsave = fig_save

### Read STELLA data

Example: https://tardis-sn.github.io/tardis/io/configuration/components/models/converters/stella_to_tardis.html

### Parameters

In [ ]:
path_mdl = './'  #

# sn_mname = ''
#sn_time, sn_mname, sn_bol = 'pyrefE5R50M26Ni2m2b5m3Z01',  -15.9  # see tt-file
# sn_time, sn_mname, sn_bol = 10, 'lbR500M20Ni01b6Z002ncVIUbox171E06',  -16.161  # see tt-file
sn_time, sn_mname, sn_bol = 20, 'lbR500M20Ni01b6Z002ncVIUbox171E06',  -15.819  # see tt-file
sn_lum = ps.MagBol2Lum(sn_bol)
print(f'{sn_mname}: Mbol= {sn_bol}  sn_lum= {sn_lum:e} ergs OR {np.log10(sn_lum/ps.phys.L_sun)}')
print(f't= {sn_time}')

fname_density_out = f'density_{sn_mname}.csv'
fname_abund_out = f'abund_{sn_mname}.csv'
print(f'{fname_density_out=}')
print(f'{fname_abund_out=}')


In [ ]:
models_info =  ps.ls.short(path_mdl)
models_data = ps.ls.long(models_info, path=path_mdl)
print(len(models_data))

In [ ]:
import h5py

def printall(name, obj):
    print(name, dict(obj.attrs))

    
fname = os.path.expanduser( os.path.join(path_mdl, sn_mname+'.h5') )

print(f'{fname=}')
    
with h5py.File(fname,'r') as hf:
    hf.visititems(printall)

In [ ]:
def h5_gr_print(gr, lvl=0):
    #Checkout what keys are inside that group.
    space = "  " * lvl
    lvl += 1
    for key in gr.keys():        
        print("{}{}".format(space, key))
        item = gr[key]
        if isinstance(item, h5py.Group):
            h5_gr_print(item, lvl)
        elif isinstance(item, h5py.Dataset):
            print("{}{} {}".format(space, key, item.shape)) 
#         else:
#             print("{}{} {}".format(space, key, type(item)))

h5data = {}

# with h5py.File(fname, "r") as f:
#     h5_gr_print(f)

In [ ]:
# fname = os.path.expanduser( os.path.join('~', sn_mname+'.h5') )
h5 = ps.H5Stella(fname)
print(fname)

hyd = h5.Hyd
hyd.Info()
print(hyd.Ntime, hyd.Nzon, hyd.Nvars, hyd.Shape, hyd.Attrs)
print(f'{hyd.R.shape=}    {hyd.T.shape=}')
t_hyd = hyd.T
print('Hyd columns: ',hyd.Columns, ' time: ', len(t_hyd))
print(f'{t_hyd=}')

In [ ]:
# nums = [0, 2, 5, 10, 20, 50]
# columns = hyd.Attrs['columns'].split()
# plt.figure(figsize=(12,12))
# for i, var in enumerate(columns):
#     for num in nums:
#         plt.subplot(2,2,i+1)
#         x = hyd.V[num, :, 0]
#         y = hyd.V[num, :, i]
#         plt.plot(x, y, 'o-', label='{} t={:.3f}'.format(str(var,'utf-8'), hyd.T[num]))
#         plt.xscale('log')
#         plt.yscale('log')
#     plt.legend()

In [ ]:

t_cur = sn_time  #time[0]
for i, t in enumerate(t_hyd):
    t_idx = i
    if t > t_cur:
        break
        
# t_idx = 10
t = t_hyd[t_idx]
print(f'{t_idx=}   {t=}')


In [ ]:
Ry = hyd.R[t_idx]  # cm
Vy = hyd.V[t_idx]  # 1000 km/s
Rhoy = hyd.Rho[t_idx]  

# Normalization
Vy = 1000 * Vy # km/s


In [ ]:
# data = stella.read_stella_data('mesa.stella.dat')


In [ ]:
# extract outer radius boundaries
# radii = data.loc[:,'outer_edge_r'].values * u.cm
radii = Ry * u.cm

# calculate t_explosion
t_explosion = t * u.day #days

# # calculate outer velocities of each zone assuming homologous expansion (v=r/t)
# velocities = (radii/t_explosion).to(u.km/u.s)

velocities = Vy *u.km/u.s

# create a new column of our velocities (OPTIONAL)
data = {'velocity':  velocities,
        'density': Rhoy,
        }

df = pd.DataFrame(data)
df

### Определение параметра time_explosion for TARDIS

$ t_{exp} = \frac{1} {\frac{dv}{dr}}$


In [ ]:
# Определение времени взрыва 
rr = Ry/1e5 * u.km  # в км/c
tt = (np.diff(velocities) / np.diff(rr))**-1 / (86400*u.s)
plt.plot(rr[:-2], tt[:-1])
plt.xlabel('Radius (km)')
plt.ylabel('1 / dv/dr (s)')
np.mean(tt[:-2])

In [ ]:
cols = {'velocity': '[cm/s]', 'density': '[g/cm^3]' }
density_dat = df[cols.keys()].reset_index(drop=True)

with open(fname_density_out, 'w') as f:
    f.write("{:.4f}  \n".format(t_explosion))
    f.write("# index  {}  \n".format(' '.join([k+' '+v  for k,v in cols.items()])))
    density_dat.to_csv(f, sep = ' ', header=False, float_format='%.6e')


In [ ]:
!head $fname_density_out

### Abundances
https://tardis-sn.github.io/tardis/io/configuration/components/models/abundancecust/abundancecust.html#csv-format

In this file:

    Header row contains element and isotopes symbol

    the remaining entries in each row give the set of elemental and isotopic abundances.

    the first column contains a running index

The abundances are specified as mass fractions (i.e. the sum of columns in each row should be 1.0). The mass fractions specified will be adopted directly in the TARDIS calculations.


In [ ]:
# todo  ADD radiactive 
eve_Zn = (1, 2, 6, 7, 8, 10,11,12,13, 14,16, 18, 20, 26, 28)
  
eve_el_m = {'H': 1.008,   'He': 4.003,  'Li': 6.941, 'Be': 9.01218, 
            'B': 10.81,   'C': 12.011,  'N': 14.007, 'O': 15.999,
            'F': 18.998,  'Ne': 20.180, 'Na': 22.990,'Mg': 24.305,
            'Al': 26.982, 'Si': 28.086, 'P': 30.974, 'S': 32.066,
            'Cl': 35.453, 'Ar': 39.948, 'K': 39.098, 'Ca': 40.078,
            'Sc': 44.956, 'Ti': 47.867, 'V': 50.942, 'Cr': 51.996,
            'Mn': 54.938, 'Fe': 55.845, 'Co': 58.933,'Ni': 58.693,
            'Cu': 63.546, 'Zn': 65.38
            }
tardis_elements = list(eve_el_m.keys())
eve_elements = ("H", "He", "C", "N", "O", "Ne", "Na", "Mg", "Al", "Si", "S", "Ar", "Ca", "Fe", "Ni")
abun = h5.Yabun

print(abun.shape)
data_abun = {nm: np.zeros(abun.shape[0]) for i, nm in enumerate(tardis_elements)}
for i, nm in enumerate(eve_elements):
    data_abun[nm] = abun[:,i]*eve_el_m[nm]
    
df_abun = pd.DataFrame(data_abun)
print(df_abun.shape)

df_abun

In [ ]:
np.sum(abun[25,:])
# df_abun.shape
# np.sum(df_abun, axis=0)

In [ ]:
# import all elements and isotopes and export to TARDIS
# data_elements = data.iloc[:,12:33].reset_index(drop=True)
cols = {'velocity': '[km/s]', 'density': '[g/cm^3]' }
# df_abun_out = df_abun[tardis_elements].reset_index(drop=True)

with open(fname_abund_out, 'w') as f:
#     f.write("{:.4f}  \n".format(t_explosion))
    f.write("# index  {}  \n".format(' '.join(tardis_elements)))
    df_abun.to_csv(f, sep = ' ', header=False, float_format='%.6e')

In [ ]:
# np.sum(list(map(float,'2.826203e-02 2.193714e-01 0.000000e+00 0.000000e+00 0.000000e+00 3.926716e-04 8.110256e-06 3.837902e-03 0.000000e+00 5.336549e-04 2.067570e-06 3.140512e-04 2.370652e-05 1.507033e-04 0.000000e+00 3.126642e-05 0.000000e+00 4.237240e-06 0.000000e+00 3.460499e-06 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 6.137989e-05 0.000000e+00 2.708673e-06 0.000000e+00 0.000000e+00'.split())))
np.sum(list(map(float,'2.848813e-02 8.781436e-01 0.000000e+00 0.000000e+00 0.000000e+00 4.716378e-03 1.136004e-04 6.140260e-02 0.000000e+00 1.076915e-02 4.753344e-05 7.633015e-03 6.396494e-04 4.232653e-03 0.000000e+00 1.002589e-03 0.000000e+00 1.692693e-04 0.000000e+00 1.386899e-04 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 3.427760e-03 0.000000e+00 1.589801e-04 0.000000e+00 0.000000e+00'.split())))

In [ ]:
!head $fname_abund_out


In [ ]:
!ls -lt | head


In [ ]:
10**(8.2+np.log10(consts.L_sun.cgs.value))
(10.12/10.)**3

In [ ]:
20 iter 10.66

Luminosity emitted   = 5.908e+41 erg / s
	Luminosity absorbed  = 3.554e+43 erg / s
	Luminosity requested = 6.095e+41 erg / s
    
40 iter 10.66
    	Current t_inner = 18341.021 K
	Expected t_inner for next iteration = 18797.566 K
 (base.py:542)
[tardis.simulation.base][INFO   ]  
	
	Starting iteration 40 of 40 (base.py:378)
[tardis.simulation.base][INFO   ]  
	
	Luminosity emitted   = 6.374e+41 erg / s
	Luminosity absorbed  = 3.533e+43 erg / s
	Luminosity requested = 6.095e+41 erg / s

In [ ]:
17659.736*0.9